In [19]:
import meshplot
import scipy as sp
import numpy as np
import igl
import meshplot as m
import gpytoolbox
import trimesh
import scipy
import rtree
import tqdm as tqdm

# Read SDF file
I generate this file from the Geometry Processing course's code. I can't release this code as the course policy, but you can have a look at the skeleton if you want: https://github.com/alecjacobson/geometry-processing-mesh-reconstruction

In [20]:

def read_matrix_from_file(filename):
    with open(filename, 'r') as file:
        line = file.readline().strip()
        x, y, z = map(int, line.split())

        total_elements = x * y * z

        positions = np.zeros((total_elements, 3)) 
        s = np.zeros(total_elements)              

        for i in range(total_elements):
            positions[i] = list(map(float, file.readline().strip().split()))

        for i in range(total_elements):
            s[i] = float(file.readline().strip())

    return x, y, z, positions, s

filename = '../data/SDF_file_elephant_high.txt'

x, y, z, positions, s = read_matrix_from_file(filename)

print(f"Dimensions: {x}, {y}, {z}")

print("Positions (first 5 entries):")
for i in range(min(5, len(positions))):
    print(f"Position {i}: {positions[i]}")

print("S values (first 5 entries):")
for i in range(min(5, len(s))):
    print(f"S[{i}] = {s[i]}")

color = s 

meshplot.plot(positions, c = color, shading={"point_size": 0.1, "colormap": "tab10"})

Dimensions: 42, 62, 56
Positions (first 5 entries):
Position 0: [-0.452293 -0.673913 -0.607253]
Position 1: [-0.430554 -0.673913 -0.607253]
Position 2: [-0.408815 -0.673913 -0.607253]
Position 3: [-0.387076 -0.673913 -0.607253]
Position 4: [-0.365337 -0.673913 -0.607253]
S values (first 5 entries):
S[0] = 0.0252924
S[1] = 0.0252922
S[2] = 0.0252918
S[3] = 0.0252912
S[4] = 0.0252905


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.006641…

In [21]:
V , F =igl.marching_cubes(s,positions,x,y,z)
meshplot.plot(V,F, shading={"point_size": 0.1, "colormap": "tab10"})

tmpmesh = trimesh.Trimesh(V,F)
query = trimesh.proximity.ProximityQuery(tmpmesh)
closest_points, distances, face_indices = query.on_surface(positions)
for i in range(len(distances)):
    if s[i]<0:
        distances[i] = distances[i] * -1
s = distances

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0107890…

In [22]:
idx = s < 0
meshplot.plot(positions[idx], c = color[idx], shading={"point_size": 0.1, "colormap": "tab10"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0042285…

In [23]:
np.min(positions, axis=0)

array([-0.452293, -0.673913, -0.607253])

In [24]:
np.max(positions, axis=0)

array([0.439011, 0.652174, 0.588399])

In [25]:
transform = np.min(positions, axis=0) * 0.5 + np.max(positions, axis=0) * 0.5 
extent = np.max(positions, axis=0) * 0.5 - np.min(positions, axis=0) * 0.5  
extent = extent * 0.9
print(transform)
print(extent)

[-0.006641  -0.0108695 -0.009427 ]
[0.4010868  0.59673915 0.5380434 ]


In [26]:
cube = trimesh.creation.box(extents=extent, transform=trimesh.transformations.translation_matrix(transform))

vertices = cube.vertices
faces = cube.faces

faces
print(np.max(vertices,axis = 0))

[0.1939024  0.28750008 0.2595947 ]


In [28]:
def barycentric_coords(point, v0, v1, v2, epsilon=1e-10):
    vec0 = v1 - v0
    vec1 = v2 - v0
    vec2 = point - v0

    d00 = np.dot(vec0, vec0)
    d01 = np.dot(vec0, vec1)
    d11 = np.dot(vec1, vec1)
    d20 = np.dot(vec2, vec0)
    d21 = np.dot(vec2, vec1)

    denom = d00 * d11 - d01 * d01

    if abs(denom) < epsilon:
        denom = 0.001

    v = (d11 * d20 - d01 * d21) / denom
    w = (d00 * d21 - d01 * d20) / denom
    u = 1.0 - v - w

    return u, v, w

def is_inside(point, v0, v1, v2):
    vec0 = v1 - v0
    vec1 = v2 - v0
    vec2 = point - v0

    return np.dot(np.cross(vec0,vec1), vec2) < 0

In [32]:
# V current position
# F faces list
# SV signed position
# S signed scalar
# local samplings
# def optimize(V,F,SV,S):

def optimization(V, F, S, SV, tau=-1, local_h=-1):

    mesh = trimesh.Trimesh(V,F)
    query = trimesh.proximity.ProximityQuery(mesh)
    closest_points, distances, face_indices = query.on_surface(SV)

    # m = len(S)
    # n = len(V)
    # A = scipy.sparse.lil_matrix((m*3,n*3))
    # T = np.zeros(m*3)

    Arows = []
    Acols = []
    Avals = []
    T = []

    local_n = 0
    for i in range(len(S)):
        if local_h > 0 and distances[i] > local_h:
            continue
        face_idx = face_indices[i]
        vid0 = F[face_idx,0]
        vid1 = F[face_idx,1]
        vid2 = F[face_idx,2]
        v0 = V[vid0]
        v1 = V[vid1]
        v2 = V[vid2]
        projection = closest_points[i]
        u, v, w = barycentric_coords(projection, v0, v1, v2)

        Arows.append(local_n*3)
        Acols.append(vid0*3)
        Avals.append(u)
        Arows.append(local_n*3)
        Acols.append(vid1*3)
        Avals.append(v)
        Arows.append(local_n*3)
        Acols.append(vid2*3)
        Avals.append(w)

        Arows.append(local_n*3+1)
        Acols.append(vid0*3+1)
        Avals.append(u)
        Arows.append(local_n*3+1)
        Acols.append(vid1*3+1)
        Avals.append(v)
        Arows.append(local_n*3+1)
        Acols.append(vid2*3+1)
        Avals.append(w)

        Arows.append(local_n*3+2)
        Acols.append(vid0*3+2)
        Avals.append(u)
        Arows.append(local_n*3+2)
        Acols.append(vid1*3+2)
        Avals.append(v)
        Arows.append(local_n*3+2)
        Acols.append(vid2*3+2)
        Avals.append(w)

        sigma = -1
        if (is_inside(SV[i],v0,v1,v2) and S[i]<0) or (not is_inside(SV[i],v0,v1,v2) and S[i] > 0):
            sigma = 1

        tmp_v = projection - SV[i]
        norm = np.linalg.norm(tmp_v)
        dir = tmp_v / norm
        ti = SV[i] + sigma*np.abs(S[i])*dir
        # print(np.linalg.norm(ti))

        T.append(ti[0])
        T.append(ti[1])
        T.append(ti[2])
        # T[local_n*3] = ti[0]
        # T[local_n*3+1] = ti[1]
        # T[local_n*3+2] = ti[2]

        local_n += 1

    m = local_n
    n = len(V)
    A = scipy.sparse.lil_matrix((m*3,n*3))
    T = np.array(T)
            
    for row, col, value in zip(Arows, Acols, Avals):
        A[row, col] = value

    I = scipy.sparse.identity(n*3, format='csr')


    V_vec = V.reshape(n*3, 1)
    T = T.reshape(m*3,1)
    # energy_ori = 0.5*np.linalg.norm(A@V_vec-T,ord=2)

    A_T_A = A.T @ A
    A_T_T = A.T @ T
    I_dense = I.toarray()

    # rho = 1/float(len(S))
    # P = -rho*A.T(A)
    # tau = 0.001

    if tau == -1:
        rho = 1 /float(len(S))
        # print(A.shape)
        # print(T.shape)
        P = -rho*A.T@(A@V_vec-T)
        AP = A@P
        # print((A@V_vec-T).shape)
        # print(AP.shape)
        tau = -(rho*np.dot((A@V_vec - T).T,AP) + 0.01*np.linalg.norm(P,ord=2)) / (rho * np.linalg.norm(AP , ord=2))
        tau = tau[0][0]
        # print(tau)
        tau = np.max([tau, 1e-6])
        tau = np.min([tau, 50])


    print(f"tau picked: {tau}")

    # tau = 0.01

    MA = I_dense + tau * A_T_A
    MB = V_vec.flatten() + tau * A_T_T.flatten()

    Vnew = np.linalg.solve(MA, MB)
    Vnew = Vnew.reshape(n*3,1)
    # energy = 1*0.5/tau*np.linalg.norm(Vnew - V_vec, ord = 2) + 0.5 *np.linalg.norm((A@Vnew - T), ord=2)
    
    # itr = 0
    # while itr < 5 and energy > energy_ori:
    #     tau = tau * 0.1
    #     MA = I_dense + tau * A_T_A
    #     MB = V_vec.flatten() + tau * A_T_T.flatten()
    #     Vtmp = np.linalg.solve(MA, MB)
    #     Vtmp = Vtmp.reshape(n,3)
    #     energy = 1*0.5/tau*(Vtmp - V_vec).T@(Vtmp - V_vec) + 0.5 *(A@V_vec - T).T*(A@V_vec - T)
    #     if energy < energy_ori:
    #         Vnew = Vtmp


    Vnew = Vnew.reshape(n,3)
    return Vnew


def reach_for_sphere(V,F,S,SV,iteration_times,h,small_iteration = 10, tau = -1, local_h = -1):
    for i in range(iteration_times):
        # mesh = trimesh.Trimesh(vertices=V,faces=F)
        # trimesh.smoothing.filter_laplacian(mesh, iterations=2)
        # V = mesh.vertices
        # F = mesh.faces
        h = h * 0.5
        print(f'current mesh size V:{V.shape[0]} F:{F.shape[0]}')
        loss = 1 
        itr = 0
        while loss > 0.00001 and itr < small_iteration:  
            V, F = gpytoolbox.remesh_botsch(V, F, 20, h, True)
            itr += 1
            V_old = V
            V = optimization(V_old,F,S,SV,tau,local_h)
            loss = np.linalg.norm((V_old - V), ord=2)
            # V, F = gpytoolbox.remesh_botsch(V, F, 20, h, True)
            print(f'iteration {i+1}, decsent times {itr}, current loss: {loss}')
        meshplot.plot(V, F, shading={"wireframe": True})
        igl.write_obj(f'{i}.obj',V,F)
        
    return V, F

In [35]:
# V = cu
# F = cg
# S = s
# V = optimization(V,F,S,positions)
# V = optimization(V,F,S,positions)
cu, cg = gpytoolbox.remesh_botsch(vertices, faces, 20, 0.1, True)
VOld = V
FOld = F


In [37]:
igl.write_obj("./input.obj",VOld,FOld)
V, F = reach_for_sphere(cu, cg, s, positions, 4, 0.1,10,tau=0.01)
meshplot.plot(V, F, shading={"wireframe": True})
igl.write_obj("./output.obj",V,F)

current mesh size V:149 F:294
tau picked: 0.01
iteration 1, decsent times 1, current loss: 0.8443945948645751
tau picked: 0.01
iteration 1, decsent times 2, current loss: 0.43542623694117377
tau picked: 0.01
iteration 1, decsent times 3, current loss: 0.31013739254356104
tau picked: 0.01
iteration 1, decsent times 4, current loss: 0.21798040276390496
tau picked: 0.01
iteration 1, decsent times 5, current loss: 0.1761386986804847
tau picked: 0.01
iteration 1, decsent times 6, current loss: 0.12928965145834606
tau picked: 0.01
iteration 1, decsent times 7, current loss: 0.10707872726061911
tau picked: 0.01
iteration 1, decsent times 8, current loss: 0.08772923488650267
tau picked: 0.01
iteration 1, decsent times 9, current loss: 0.08153632132831196
tau picked: 0.01
iteration 1, decsent times 10, current loss: 0.06815929528267509


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0110295…

current mesh size V:856 F:1708
tau picked: 0.01
iteration 2, decsent times 1, current loss: 0.05509945167029123
tau picked: 0.01
iteration 2, decsent times 2, current loss: 0.0425686088551329
tau picked: 0.01
iteration 2, decsent times 3, current loss: 0.03911676167772746
tau picked: 0.01
iteration 2, decsent times 4, current loss: 0.02923338417720112
tau picked: 0.01
iteration 2, decsent times 5, current loss: 0.026414688553354645
tau picked: 0.01
iteration 2, decsent times 6, current loss: 0.02478028260420863
tau picked: 0.01
iteration 2, decsent times 7, current loss: 0.02270318768484096
tau picked: 0.01
iteration 2, decsent times 8, current loss: 0.022575495419947033
tau picked: 0.01
iteration 2, decsent times 9, current loss: 0.02069864107595857
tau picked: 0.01
iteration 2, decsent times 10, current loss: 0.01996130821677107


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0103570…

current mesh size V:3439 F:6874


KeyboardInterrupt: 